# ** 多输入通道和多输出通道**

In [1]:
import torch
import sys
sys.path.append(r'C:\D\ProgramFile\jupyter\torch_learn\dive_to_dp\utils')
import d2lzh as d2l

## **多输入通道**

In [2]:
def corr2d_multi_in(X, K):
    res = d2l.corr2d(X[0, :, :], K[0, :, :])
    for i in range(1, X.shape[0]):
        res += d2l.corr2d(X[i, :, :], K[i, :, :])
    return res

In [9]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

In [4]:
X

tensor([[[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]],

        [[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])

In [5]:
K

tensor([[[0, 1],
         [2, 3]],

        [[1, 2],
         [3, 4]]])

In [6]:
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## **多通道输出**

假设K是一个四维数组，第一维上面的每一个二维矩阵都是以卷积核，在计算后需要把各个二维向量组合起来再变成三维

In [7]:
def corr2d_multi_in_out(X, K):
    # 对K的第0维遍历，每次同输入X做互相关计算。所有结果使用stack函数合并在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K])

stack函数默认在第一维,注意stack和cat的区别，stack会添加一维

In [10]:
K = torch.stack([K, K+1, K+2], dim=0)
K.shape

torch.Size([3, 2, 2, 2])

In [11]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## **$1 \times 1$ 卷积**

一维卷积的计算基本都在通道层上，主要作用是降低通道数

In [12]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.view(c_i, h * w)
    K = K.view(c_o, c_i)
    Y = torch.mm(K, X)
    return Y.view(c_o, h, w)

In [14]:
X = torch.randn(3, 3, 3)
K = torch.rand(2, 3, 1, 1)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().item() < 1e-6

True